**Experiments with Mistral7B-Instruct (baseline) and fine tuned model with Oscard Wilde texts**

The idea of the experiments consists in using the Classifier trained for authorship attribution (BertAA_Wilde_vs_Mistral) to test the capacity of the fine tuned model to mimetize Wilde's writting style.

Then, for each model we are going to generate fiction stories from the same starting lines and assign labels to each story and compare the results between the two models. The hypothesis is that for the fine tuned model the capacity of the model to distinguish between Wilde and Mistral will decay.

In [ ]:
# uncomment the following lines to run in google colab
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# install the required packages if working in google colab
# !pip install -U simpletransformers
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q trl xformers wandb datasets einops gradio sentencepiece

In [ ]:
# import the required packages
import torch
import simpletransformers
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, platform, gradio, warnings
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from huggingface_hub import notebook_login
import json
from tqdm import tqdm
import utils

In [ ]:
# data and models paths
dir_root = '.' # comment this line to run in google colab
# dir_root = './drive/MyDrive/DL-ENS' # uncomment this line to run in google colab
dir_data = f'{dir_root}/dataset'
models_path = f'{dir_root}/models'
clf_path = f'{models_path}/BERTAA_wilde_vs_mistral7B.pt'
list_to_generate_path = f'{dir_data}/story_prompts_for_evaluation.txt'
ft_model = f'{dir_root}/models/Mistral7B_fine_tuned_OscarWilde.pt'

In [ ]:
# load classifier (wilde vs mistral7B-Instruct)
clf = torch.load(clf_path)
clf

In [ ]:
#base model
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

In [ ]:
# load the lines to start the generation
texts_to_generation = []
with open(list_to_generate_path, 'r+', encoding='utf-8') as fd:
  texts_to_generation = fd.readlines()
texts_to_generation = [text[:-1] for text in texts_to_generation]

**Experiments with baseline model**

In [ ]:
model = load_model(model_name)
tokenizer = load_tokenizer(model_name)

In [ ]:
author_preds, generated_texts = clf_exp(model, tokenizer, clf, texts_to_generation)

In [7]:
print(f"Predictions:\n Wilde: {author_preds.count(0)/len(author_preds) * 100} % \n Mistral7B: {author_preds.count(1)/len(author_preds) * 100} %")

In [ ]:
# saver the results
save_generated_texts(generated_texts, author_preds, model = 'baseline', data_path = dir_data)

**Experiments with fine-tuned model**

In [ ]:
model = load_model(model_name, adapt = True, from_fine_tuned = True, model_path = ft_model)
tokenizer = load_tokenizer(model_name)

In [ ]:
author_preds_ft, generated_texts_ft = clf_exp(model, tokenizer, clf, texts_to_generation)

In [ ]:
print(f"Predictions:\n Wilde: {author_preds_ft.count(0)/len(author_preds_ft) * 100} % \n Mistral7B: {author_preds_ft.count(1)/len(author_preds_ft) * 100} %")

In [ ]:
# saver the results
save_generated_texts(generated_texts_ft, author_preds_ft, model = 'Mistral7B_fine_tuned', data_path = dir_data)

In [8]:
print("Done")

Done
